### Exercise 1
Suppose the people who own page 3 in the web of Figure 1 are infuriated by the
fact that its importance score, computed using formula (2.1), is lower than the score of page 1. In
an attempt to boost page 3’s score, they create a page 5 that links to page 3; page 3 also links to page
5. Does this boost page 3’s score above that of page 1?

![image](./img/image.png)

In [2]:
import numpy as np

A = np.array([[0, 0, 1, 1/2],
         [1/3, 0, 0, 0],
         [1/3, 1/2, 0, 1/2],
         [1/3, 1/2, 0, 0]])

A_mod = np.array([[0, 0, 1/2, 1/2, 0],
         [1/3, 0, 0, 0, 0],
         [1/3, 1/2, 0, 1/2, 1],
         [1/3, 1/2, 0, 0, 0],
         [0, 0, 1/2, 0, 0]])

#print(A)
print(np.array2string(A, formatter={'float_kind':lambda x: "%.3f" % x}))
#print(A_mod)
print(np.array2string(A_mod, formatter={'float_kind':lambda x: "%.3f" % x}))

[[0.000 0.000 1.000 0.500]
 [0.333 0.000 0.000 0.000]
 [0.333 0.500 0.000 0.500]
 [0.333 0.500 0.000 0.000]]
[[0.000 0.000 0.500 0.500 0.000]
 [0.333 0.000 0.000 0.000 0.000]
 [0.333 0.500 0.000 0.500 1.000]
 [0.333 0.500 0.000 0.000 0.000]
 [0.000 0.000 0.500 0.000 0.000]]


In [3]:
#Stockastic check
def isSto(A): 
    col_sums = A_mod.sum(axis=0)
    return (col_sums == 1).all()

if isSto(A):
    print("A is stochastic")
if isSto(A_mod):
    print("A_mod is stochastic")


A is stochastic
A_mod is stochastic


In [4]:

def print_eig(label, eigvals, eigvecs):
    print(f"\n=== Eigen Decomposition for {label} ===")
    for i, (val, vec) in enumerate(zip(eigvals, eigvecs.T), 1):
        print(f"Eigenvalue {i}: {val:.6f}")
        print(f"Eigenvector {i}: {vec}")
        print("-" * 40)


eigvals_A, eigvecs_A = np.linalg.eig(A)
eigvals_Am, eigvecs_Am = np.linalg.eig(A_mod)

#print_eig("A", eigvals_A, eigvecs_A)
#print_eig("A_mod", eigvals_Am, eigvecs_Am)

# Function
def standardizer(x, round=3):
    sum = x.sum()
    x = x/sum
    return np.round(x, round)


index = np.where(np.isclose(eigvals_A, 1.0, atol=1e-8)) # we cannot use np.where because of rounding errors
print(np.real(standardizer(eigvecs_A.T[index])))
index = np.where(np.isclose(eigvals_Am, 1.0, atol=1e-8))
print(np.real(standardizer(eigvecs_Am.T[index])))

[[0.387 0.129 0.29  0.194]]
[[0.245 0.082 0.367 0.122 0.184]]


## Exercise 4
In the web of Figure 2.1, remove the link from page 3 to page 1. In the resulting
web page 3 is now a dangling node. Set up the corresponding substochastic matrix and find its largest
positive (Perron) eigenvalue. Find a non-negative Perron eigenvector for this eigenvalue, and scale
the vector so that components sum to one. Does the resulting ranking seem reasonable?

In [5]:
B = np.array([[0, 0, 0, 1/2],
         [1/3, 0, 0, 0],
         [1/3, 0, 0, 1/2],
         [1/3, 1, 0, 0]])

In [11]:
def get_perron(matrix):
    A = np.array(matrix, dtype = float)
    eigvals, eigvecs = np.linalg.eig(matrix)
    idx = np.argmax(np.abs(eigvals))

    perron_eigenvalue = eigvals[idx].real
    perron_eigenvector = eigvecs[:, idx].real

        # Make the eigenvector non-negative (flip sign if needed)
    if perron_eigenvector[0] < 0:
        perron_eigenvector = -perron_eigenvector
    
    # Normalize to sum to 1 (or L2 norm = 1, depending on preference)
    perron_eigenvector = standardizer(perron_eigenvector)
    
    return perron_eigenvalue, perron_eigenvector

print(get_perron(B))

(np.float64(0.6503404456767778), array([0.231, 0.118, 0.35 , 0.301]))


This substochastic matrix representing the web described previously, lead to a rank as follow:
- Page 3
- Page 4
- Page 1
- Page 2

This does make sense because a web user after a finite number of 'random click' would reach the page 3. After that it would be stuck because page 3 does not link to any other page.